In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [26]:
car_train=pd.read_csv("../data/train.csv")
car_test=pd.read_csv("../data/test.csv")

In [27]:
#since the discount column is missing in both train and test set lets remove that column
car_train = car_train.drop('Descuento',axis=1)
# Probar a rellenar
car_train = car_train.dropna()

In [28]:
car_train['Consumo']=car_train.Consumo.astype(str).str.partition()[0].astype('float')
car_train['Motor_CC']=car_train.Motor_CC.astype(str).str.partition()[0].astype('int')
car_train['Potencia']=car_train.Potencia.astype(str).str.partition()[0].astype('float')
car_train['Asientos']=car_train.Asientos.astype('int')
part = car_train['Nombre'].astype(str).str.partition()
car_train['Company'] = part[0].str.title()
#car_train_2['Car_name'] = part[2]
car_train.drop(['Nombre'], axis = 1, inplace = True)

In [29]:
car_train.replace({'First':1,'Second':2,'Third':3,'Fourth & Above':4},inplace = True)

In [30]:
tipo_marchas = car_train[['Tipo_marchas']]
tipo_marchas = pd.get_dummies(tipo_marchas, drop_first=True)
ciudad = car_train[['Ciudad']]
ciudad = pd.get_dummies(ciudad, drop_first=True)
company = car_train[['Company']]
company = pd.get_dummies(company, drop_first=True)
combustible = car_train[['Combustible']]
combustible = pd.get_dummies(combustible, drop_first=True)

In [31]:
car_train = pd.concat([car_train, company , ciudad , tipo_marchas, combustible], axis = 1)

In [32]:
car_train.drop(['id','Ciudad', 'Combustible', 'Tipo_marchas', 'Company'], axis = 1, inplace = True)

In [33]:
car_train.head()

,Año,Kilometros,Mano,Consumo,Motor_CC,Potencia,Asientos,Precio_cat,Company_Bentley,Company_Bmw,...,Ciudad_G,Ciudad_H,Ciudad_I,Ciudad_J,Ciudad_K,Ciudad_L,Tipo_marchas_Manual,Combustible_Diesel,Combustible_LPG,Combustible_Petrol
0,2012.0,83000.0,1,23.40,1248,74.00,5,3,0,0,...,1,0,0,0,0,0,1,1,0,0
1,2016.0,4800.0,1,20.51,998,67.04,5,3,0,0,...,0,0,1,0,0,0,1,0,0,1
2,2016.0,26000.0,1,25.32,1198,77.00,6,3,0,0,...,0,0,0,0,0,0,1,1,0,0
3,2013.0,56127.0,1,18.50,1197,80.00,5,3,0,0,...,0,0,0,0,0,0,1,0,0,1
4,2017.0,41981.0,1,18.70,1199,88.70,5,3,0,0,...,0,1,0,0,0,0,1,0,0,1


In [45]:
car_train.columns

Index(['Año', 'Kilometros', 'Mano', 'Consumo', 'Motor_CC', 'Potencia',
       'Asientos', 'Precio_cat', 'Company_Bentley', 'Company_Bmw',
       'Company_Chevrolet', 'Company_Datsun', 'Company_Fiat', 'Company_Force',
       'Company_Ford', 'Company_Honda', 'Company_Hyundai', 'Company_Isuzu',
       'Company_Jaguar', 'Company_Jeep', 'Company_Lamborghini', 'Company_Land',
       'Company_Mahindra', 'Company_Maruti', 'Company_Mercedes-Benz',
       'Company_Mini', 'Company_Mitsubishi', 'Company_Nissan',
       'Company_Porsche', 'Company_Renault', 'Company_Skoda', 'Company_Tata',
       'Company_Toyota', 'Company_Volkswagen', 'Company_Volvo', 'Ciudad_C',
       'Ciudad_D', 'Ciudad_E', 'Ciudad_F', 'Ciudad_G', 'Ciudad_H', 'Ciudad_I',
       'Ciudad_J', 'Ciudad_K', 'Ciudad_L', 'Tipo_marchas_Manual',
       'Combustible_Diesel', 'Combustible_LPG', 'Combustible_Petrol'],
      dtype='object')

In [35]:
car_test = car_test.drop('Descuento',axis=1)

In [34]:
car_test['Consumo']=car_test.Consumo.astype(str).str.partition()[0].astype('float')
car_test['Motor_CC']=car_test.Motor_CC.astype(str).str.partition()[0].astype('int')
car_test['Potencia']=car_test.Potencia.astype(str).str.partition()[0].astype('float')
car_test['Asientos']=car_test.Asientos.astype('int')
part = car_test['Nombre'].astype(str).str.partition()
car_test['Company'] = part[0].str.title()
#car_train_2['Car_name'] = part[2]
car_test.drop(['Nombre'], axis = 1, inplace = True)

In [36]:
tipo_marchas = car_test[['Tipo_marchas']]
tipo_marchas = pd.get_dummies(tipo_marchas, drop_first=True)
ciudad = car_test[['Ciudad']]
ciudad = pd.get_dummies(ciudad, drop_first=True)
company = car_test[['Company']]
company = pd.get_dummies(company, drop_first=True)
combustible = car_test[['Combustible']]
combustible = pd.get_dummies(combustible, drop_first=True)

In [38]:
car_test.drop(['Ciudad', 'Combustible', 'Tipo_marchas', 'Company'], axis = 1, inplace = True)

In [39]:
# Get missing columns in the training test
missing_cols = set( car_train.columns ) - set( car_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    car_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
car_test = car_test[car_train.columns]

In [40]:
# load dataset
X = car_train.drop('Precio_cat', axis=1)
Y = car_train['Precio_cat']

In [41]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [48]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=48, activation='relu'))
	model.add(Dense(5, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [49]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 24.76% (13.68%)
